In [495]:
from pymnet import *
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import mglearn
import math
import random
import networkx as nx
from sympy import *      
from mpl_toolkits.mplot3d import Axes3D
import numpy as np        
from matplotlib import cm
import pylab
import time

In [623]:
def A_layer_config(state, node, edge, Max, Min) :   # A_layer 구성요소 A_layer_config(state = [1,2], node = 2048, edge = 5, Max = 2, Min = -2)
    global A_state, A_node, A_edge, MAX, MIN
    A_state = state                                 # state = [   ]  kinds of states
    A_node = node
    A_edge = edge
    MAX = Max
    MIN = Min
    return A_state, A_node, A_edge, MAX, MIN
    
def B_layer_config(state, node, edge, inter_edge) : # B_layer 구성요소 B_layer_config(state = [-1], node = 2048, edge = 5, inter_edge= 1)
    global B_state, B_node, B_edge, inter_edges
    B_state = state                                 # state = [   ]  kinds of states
    B_node = node
    B_edge = edge
    inter_edges = inter_edge  
    return B_state, B_node, B_edge, inter_edges

def making_layer_A_random_regular() :               # A_layer random_regular network
    global A, A_edges
    A = np.array(A_state * int(A_node/len(A_state)), int)
    random.shuffle(A)
    A_edges = nx.random_regular_graph(A_edge, A_node, seed=None)
    return A, A_edges, MAX, MIN

def making_layer_B_random_regular() :               # B_layer random_regular network
    global B, B_edges
    B = np.array(B_state * int(B_node/len(B_state)), int)
    random.shuffle(B)
    B_edges = nx.random_regular_graph(B_edge, B_node, seed=None)
    return B, B_edges

def making_layer_A_barabasi_albert() :     # A_layer 바바라시-알버트 네트워크
    global A, A_edges
    A = np.array(A_state * int(A_node/len(A_state)), int)
    random.shuffle(A)
    A_edges = nx.barabasi_albert_graph(A_node, A_edge, seed=None)
    return A, A_edges, MAX, MIN

def making_layer_B_barabasi_albert() :     # B_layer 바바라시-알버트 네트워크
    global B, B_edges
    B = np.array(B_state * int(B_node/len(B_state)), int)
    random.shuffle(B)
    B_edges = nx.barabasi_albert_graph(B_node, B_edge, seed=None)
    return B, B_edges

def making_interconnected_edges() :        # 
    global AB_edges, AB_neighbor
    AB_edges = []
    AB_neighbor = []
    for i in range(int(A_node/inter_edges)) :
        for j in range(inter_edges) :
            connected_A_node = np.array(A_edges.nodes).reshape(-1, inter_edges)[i][j]
            AB_neighbor.append(connected_A_node)
            AB_edges.append((i, connected_A_node))
    AB_neighbor = np.array(AB_neighbor).reshape(-1, inter_edges)
    return AB_edges, AB_neighbor

def select_layer_A_model(a) :
    if a == 1 :
        making_layer_A_random_regular()
    elif a == 2 :
        making_layer_A_barabasi_albert()
    return A, A_edges, MAX, MIN
        
    
def select_layer_B_model(b) :
    if b == 1 :
        making_layer_B_random_regular()
    elif b == 2 :
        making_layer_B_barabasi_albert()
    return B, B_edges


def making_A_layer_graph() :
    global A_pair, A_nodecolor
    A_pair = []
    A_nodecolor = []
    inter_net.add_layer('A_layer')
    for i in sorted(A_edges.nodes) :
        inter_net.add_node(i)
    for i, j in sorted(A_edges.edges) :
        inter_net[i, j, 'A_layer'] = 1
        A_pair.append(((i,'A_layer'),(j,'A_layer')))
        if A[i] == 2 :
            A_nodecolor.append((i, 'A_layer', 2))
        elif A[i] == 1 :
            A_nodecolor.append((i, 'A_layer', 1))
        if A[j] == 2 :
            A_nodecolor.append((j, 'A_layer', 2))
        elif A[j] == 1 :
            A_nodecolor.append((j, 'A_layer', 1))
    return inter_net

# "rule":"edgeweight","colormap":"jet","scaleby":0.1

def making_B_layer_graph() :
    global B_pair, B_nodecolor
    B_pair = []
    B_nodecolor = []
    inter_net.add_layer('B_layer')
    for i in sorted(B_edges.nodes) :
        inter_net.add_node(i)
    for i, j in sorted(B_edges.edges) :
        inter_net[i, j, 'B_layer'] = 1
        B_pair.append(((i,'B_layer'),(j,'B_layer')))
        if B[i] == 1 :
            B_nodecolor.append((i, 'B_layer', 1))
        elif B[i] == -1 :
            B_nodecolor.append((i, 'B_layer', -1))
        if B[j] == 1 :
            B_nodecolor.append((j, 'B_layer', 1))
        elif B[j] == -1 :
            B_nodecolor.append((j, 'B_layer', -1))
    return inter_net

def inter_edge_graph() :
    global inter_pair
    inter_pair = []
    global inter_net
    inter_net = MultilayerNetwork(aspects = 1)
    making_A_layer_graph()
    making_B_layer_graph()
    for i, j in sorted(AB_edges) :
        inter_net[j, 'A_layer'][i, 'B_layer'] = 1
        inter_pair.append(((j,'A_layer'),(i,'B_layer')))
    return inter_net

def edgecolordic() :
    global edgeColorDict
    edgeColorDict={}
    for i in range(int(len(A_edges.edges))) :
        edgeColorDict[A_pair[i]] = 'r'
    for i in range(int(len(B_edges.edges))) :
        edgeColorDict[B_pair[i]] = 'b'
    for i in range(int(len(AB_edges))) :
        edgeColorDict[inter_pair[i]] = 'g'
    return edgeColorDict

## nodeColorDict={(0,0):"r",(1,0):"r",(0,1):"r"}
def nodecolordic() :
    global nodeColorDict
    nodeColorDict={}
    for i in range(int(len(A_nodecolor))) :
        if A_nodecolor[i][2] == 2 :
            nodeColorDict[(A_nodecolor[i][0], 'A_layer')] = 'red'
        elif A_nodecolor[i][2] == 1 :
            nodeColorDict[(A_nodecolor[i][0], 'A_layer')] = 'hotpink'
    for i in range(int(len(B_nodecolor))) :
        if B_nodecolor[i][2] == 1 :
            nodeColorDict[(B_nodecolor[i][0], 'B_layer')] = 'hotpink'
        elif B_nodecolor[i][2] == -1 :
            nodeColorDict[(B_nodecolor[i][0], 'B_layer')] = 'deepskyblue'            
    return nodeColorDict

def plot_3D(result, a, b, c, d, e) :       # ex__  plot_3D('result11_data2.pickle', 'ganma', 'beta', 'B layer mean', 45, 45)
    sns.set_style("whitegrid")
    final_data = pd.read_pickle(result)
    ax=plt.axes(projection='3d')
    ax.plot_trisurf(final_data[a], final_data[b], final_data[c], cmap='RdBu', edgecolor='none')
    ax.set_xlabel(str(a))
    ax.set_ylabel(str(b))
    ax.set_zlabel(str(c))
    ax.view_init(d, e)
    # ax.scatter(final_data[a], final_data[b], final_data[c], c=final_data[c], cmap='RdBu', linewidth=0.1) ax.view_init(d, e)

    
def plot_2D(result, a, b, c, d, e) :       # ex__  plot_2D('result2.3_data.pickle', 'beta', 'B layer mean', 'ganma', 1.0, 'ganma = 1')
    sns.set_style("whitegrid")
    final_data = pd.read_pickle(result)
    mini = final_data[final_data[c] > (d - 0.001)]
    two_dimension = mini[mini[c] < (d + 0.001)]
    plt.plot(two_dimension[a], two_dimension[b], '-', label = e)
    plt.legend(framealpha = 1, frameon = True)
    plt.ylim(-1.1, 1.1)
    plt.xlabel(str(a))
    plt.ylabel(str(b))

In [656]:
A_layer_config(state = [1,2], node = 32, edge = 3, Max = 2, Min = -2)
B_layer_config(state = [-1], node = 8, edge = 3, inter_edge= 4)
A_layer_model = select_layer_A_model(2)
B_layer_model = select_layer_B_model(2)
making_interconnected_edges()

([(0, 0),
  (0, 1),
  (0, 2),
  (0, 3),
  (1, 4),
  (1, 5),
  (1, 6),
  (1, 7),
  (2, 8),
  (2, 9),
  (2, 10),
  (2, 11),
  (3, 12),
  (3, 13),
  (3, 14),
  (3, 15),
  (4, 16),
  (4, 17),
  (4, 18),
  (4, 19),
  (5, 20),
  (5, 21),
  (5, 22),
  (5, 23),
  (6, 24),
  (6, 25),
  (6, 26),
  (6, 27),
  (7, 28),
  (7, 29),
  (7, 30),
  (7, 31)],
 array([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23],
        [24, 25, 26, 27],
        [28, 29, 30, 31]]))

In [657]:
inter_edge_graph()
fig = draw(inter_net, show=False, layout="spring", layergap=1.3,layerPadding=0.05, alignedNodes=True, ax=None, 
           layerColorDict={'A_layer' :'pink', 'B_layer' :'steelblue'}, layerColorRule={}, defaultLayerColor='#29b7c1',
           edgeColorDict= edgecolordic(),
           edgeWidthDict={}, edgeWidthRule={}, defaultEdgeWidth=0.7, 
           edgeStyleDict={}, edgeStyleRule={'inter': ':', 'intra': '-', 'rule': 'edgetype'}, defaultEdgeStyle='-',
           nodeLabelDict= {}, nodeLabelRule={}, defaultNodeLabel=None, 
           nodeColorDict= nodecolordic(), nodeColorRule={}, defaultNodeColor=None,
           nodeLabelColorDict = {} ,nodeLabelColorRule={}, defaultNodeLabelColor='k',
           nodeSizeDict={}, nodeSizeRule={'scalecoeff': 0.3, 'rule': 'scaled'}, defaultNodeSize=None)
fig.savefig("inter_net_BA_leader.pdf")

C:\Users\Purple\Anaconda3\lib\site-packages\matplotlib\pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [644]:
A_layer_config(state = [1,2], node = 2048, edge = 5, Max = 2, Min = -2)
B_layer_config(state = [-1], node = 2048, edge = 5, inter_edge= 1)
A_layer_model = select_layer_A_model(1)
B_layer_model = select_layer_B_model(1)

In [647]:
len(sorted(B_edges.edges))

5120

In [648]:
len(sorted(B_edges.edges))

5120

In [624]:
plot_3D('result6.1_data.pickle', 'ganma', 'beta', 'B layer mean',30, 120)

FileNotFoundError: [Errno 2] No such file or directory: 'result6.1_data.pickle'